In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
%aimport data_processing
%aimport codelists
%aimport constants
%aimport import_file

In [ ]:
from data_processing import *

In [ ]:
from import_file import *

In [ ]:
pd.options.display.max_rows = 200

In [ ]:
# clean_matching()

In [ ]:
# create_pegmed()

In [ ]:
# create_pegprod()

In [ ]:
# create_medcoded_entries()

In [ ]:
# create_prescriptions()

In [ ]:
# create_consultations()

In [ ]:
# all_entries = create_all_entries()

In [ ]:
all_entries = pd.read_csv('data/pt_data/all_entries.csv',delimiter=',',parse_dates=['sysdate','eventdate'],infer_datetime_format=True)

In [ ]:
# all_entries.sample(5)

In [ ]:
create_pt_features()

In [ ]:
# get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries)

In [ ]:
# add_data_start_and_end_dates(all_entries)

In [ ]:
# calculate_amount_of_data_available(isCase=True)

In [ ]:
# delete_patients_if_not_enough_data(isCase=True)

In [ ]:
# match_cases_and_controls()

In [ ]:
# delete_unmatched_cases_and_controls()

In [ ]:
# calculate_amount_of_data_available(isCase=False)

In [ ]:
# get_insomnia_event_count()